In [312]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [827]:
from collections import defaultdict
from collections.abc import Iterable
import gc
import inspect
import matplotlib.pyplot as plt
import numpy as np
from operator import gt, lt, add, sub
import os
import pandas as pd
from tabulate import tabulate
from sklearn.metrics import (accuracy_score, dcg_score, roc_auc_score, 
                             precision_score, recall_score)
from textblob import TextBlob
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import Adam
import warnings

from accio.s3tool import S3tool
from htools import hdir, LoggerMixin, eprint, assert_raises, auto_repr
from ml_htools.torch_utils import (ModelMixin, variable_lr_optimizer,
                                   update_optimizer, DEVICE, stats, adam)
from spellotape.utils import stop_instance

In [314]:
# Reproducible testing.
np.random.seed(0)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

# To Do:

- Maybe get different logger and make new folder and/or file for each training run?
- Finish + test csvlogger (decide whether to comebine with statshandler)
- Build + add + test LRScheduler
- Test regression
- s3 upload callback
- Handle case when softmax needed on outputs.

In [4]:
class Data(Dataset):
    
    def __init__(self, n=64, dim=2):
        self.x = torch.rand(n, dim).float()
        self.y = torch.clamp(
            (self.x[:, 0]*.75 + self.x[:, 1]*.25).round(), 0, 1
        ).abs().unsqueeze(-1)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return len(self.x)

In [875]:
class Trainer(LoggerMixin):
    
    def __init__(self, net, ds_train, ds_val, dl_train, dl_val,
                 criterion, out_dir, bucket=None, optim=Adam, eps=1e-3, 
                 last_act=None, threshold=0.5,
                 metrics=None, callbacks=None, device=DEVICE):
        """
        Parameters
        ----------
        last_act: callable or None
            Last activation function to be applied outside the model. 
            For example, for a binary classification problem, if we choose
            to use binary_cross_entropy_with_logits loss but want to compute
            some metric using soft predictions, we would pass in torch.sigmoid
            for last act. For a multi-class problem using F.cross_entropy loss,
            we would need to pass in F.softmax to compute predicted 
            probabilities.  Remember this is ONLY necessary if all of the 
            following conditions are met:
            1. It is a classification problem.
            2. We have excluded the final activation from our model for 
            numerical stability reasons. (I.E. the loss function has the 
            the final activation built into it.)
            3. We wish to compute 1 or more metrics based on soft predictions,
            such as AUC-ROC.
        optim: torch.optim callable
            Callable optimizer. The default is Adam.
        threshold: float or None
            For a classification problem, pass in the decision threshold to
            use when converting soft predictions to hard predictions. For a
            regression problem, pass in None.
        """
        self.net = net
        self.ds_train, self.ds_val = ds_train, ds_val
        self.dl_train, self.dl_val = dl_train, dl_val
        
        # LR(s) will be updated in fit() method.
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            self.optim = variable_lr_optimizer(net, optimizer=optim, eps=eps)
        self.criterion = criterion
        self.device = DEVICE
        self.last_act = last_act
        self.thresh = threshold
        self._stop_training = False
        self.logger = None
    
        # Storage options.
        self.out_dir = out_dir
        self.bucket = bucket
        os.makedirs(out_dir, exist_ok=True)
        
        # Dict makes it easier to adjust callbacks after creating model.
        self.callbacks = {}
        self.add_callbacks(*[BasicConfig(), StatsHandler(), MetricPrinter()] 
                           + (callbacks or []))
        self.metrics = [batch_size] + (metrics or [])
    
    def save(self, fname):
        save(self, os.path.join(self.out_dir, fname))
        
    def load(self, fname):
        """This lets a trainer load a previously saved state. This is NOT
        an in-place operation: the new trainer is simply returned.
        
        Parameters
        ----------
        fname: str
            Name of file where Trainer object is stored. Must end in either
            .zip or .pkl. Do not include the full path. This automatically
            checks the output directory.
        
        Examples
        --------
        trainer = Trainer(...)
        trainer.fit(...)
        trainer.save('v1')
        trainer = trainer.load('v1')
        """
        return load(self, os.path.join(self.out_dir, fname))
    
    @staticmethod
    def from_file(path):
        """Ths lets us load a previously saved Trainer. Unlike load(), this
        does not require us to have a Trainer instance first. The intent is
        that load() works well when within a single Jupyter notebook session,
        but when returning to work on a different day, we may not have a live
        instance of Trainer and a staticmethod lets us load without
        remembering all the arguments used at initialization time.
        
        Parameters
        ----------
        path: str
            Full path to saved file. This differs from load() because here, we
            don't have an instance with an out_dir attribute to check.
        """
        return load(path)
    
    def add_callbacks(self, *callbacks):
        """
        Parameters
        ----------
        callbacks: TorchCallback
            One or more callbacks to append to 
        """
        self.callbacks.update({type(cb).__name__: cb for cb in callbacks})
        self.callbacks = dict(sorted(self.callbacks.items(),
                                     key=lambda x: x[1].priority))
    
    def add_metrics(self, *metrics):
        self.metrics.extend(metrics)
    
    def fit(self, epochs, lrs=3e-3): 
        stats = defaultdict(list)
        sum_i = 0
        _ = self.decide_stop_training('on_train_begin', lrs)
        for e in range(1, epochs+1):
            _ = self.decide_stop_training('on_epoch_begin', e, stats, None)
            for i, batch in enumerate(self.dl_train, 1):
                sum_i += 1
                *xb, yb = map(lambda x: x.to(self.device), batch)
                self.optim.zero_grad()
                _ = self.decide_stop_training('on_batch_begin', i, sum_i, stats)
                
                # Forward and backward passes.
                y_score = self.net(*xb)
                loss = self.criterion(y_score, yb)
                loss.backward()
                self.optim.step()
                
                # Separate because callbacks are only applied during training.
                self._update_stats(stats, loss, yb, y_score.detach())
                if self.decide_stop_training('on_batch_end', i, sum_i, stats): 
                    break
            
            # If on_batch_end callback halts training, else block is skipped.  
            else: 
                val_stats = self.validate()
                if self.decide_stop_training('on_epoch_end', e, stats, val_stats):
                    break
                continue
            break      

        _ = self.decide_stop_training('on_train_end', e, stats, val_stats)
    
    def validate(self, dl_val=None):
        # Allow user to pass in different loader outside of training.
        dl_val = self.dl_val or dl_val
        val_stats = defaultdict(list)
        self.net.eval()
        with torch.no_grad():
            for batch in dl_val:
                *xb, yb = map(lambda x: x.to(self.device), batch)
                y_score = self.net(*xb)
                loss = self.criterion(y_score, yb)
                self._update_stats(val_stats, loss, yb, y_score)
        return val_stats
        
    def _update_stats(self, stats, loss, yb, y_score):
        """Update stats in place.
        
        Parameters
        ----------
        stats: defaultdict[str, list]
        loss: torch.Tensor
            Tensor containing single value (mini-batch loss).
        yb: torch.Tensor
            Mini-batch of labels.
        y_pred: torch.Tensor
            Mini-batch of predictions.
            
        Returns
        -------
        None
        """
        try:
            y_score = self.last_act(y_score)
        except TypeError:
            pass
        y_pred = (y_score > self.thresh).float() if self.thresh else y_score
            
        stats['loss'].append(loss.detach().cpu().numpy().item())
        for m in self.metrics:
            yhat = y_pred if hasarg(m, 'y_pred') else y_score
            stats[m.__name__.replace('_score', '')].append(m(yb, yhat))
        
    def decide_stop_training(self, attr, *args, **kwargs):
        self._stop_training = False
        # Pass model object as first argument to callbacks.
        for cb in self.callbacks.values():
            getattr(cb, attr)(self, *args, **kwargs)
        return self._stop_training
    
    def unfreeze(self, n_layers=None, n_groups=None):
        """Pass in either the number of layers or number of groups to 
        unfreeze. Unfreezing always starts at the end of the network and moves
        backward (e.g. n_layers=1 will unfreeze the last 1 layer, or n_groups=2 
        will unfreeze the last 2 groups.) Remember than weights and biases are 
        treated as separate layers.
        """
        self.net.unfreeze(n_layers, n_groups)
            
    def freeze(self):
        """Freeze whole network."""
        self.net.unfreeze(n_layers=0)
    
    def __repr__(self):
        r = (f'Trainer(criterion={repr(self.criterion.__name__)}, '
             f'out_dir={repr(self.out_dir)}, bucket={repr(self.bucket)})'
             f'\n\nDatasets: {len(self.ds_train)} train rows, '
             f'{len(self.ds_val)} val rows'
             f'\n\nOptimizer: {repr(self.optim)}'
             f'\n\n{repr(self.net)})')
        return r

In [758]:
class BaseModel(nn.Module):
    
    def unfreeze(self, n_layers=None, n_groups=None):
        """Pass in either the number of layers or number of groups to 
        unfreeze. Unfreezing always starts at the end of the network and moves
        backward (e.g. n_layers=1 will unfreeze the last 1 layer, or n_groups=2 
        will unfreeze the last 2 groups.) Remember than weights and biases are 
        treated as separate layers.
        """
        if n_groups is not None: 
            self._unfreeze_by_group(n_groups)
            return

        length = len(self)
        for i, p in enumerate(self.parameters()):
            p.requires_grad = i >= length - n_layers
            
    def freeze(self):
        """Freeze whole network."""
        self.unfreeze(n_layers=0)
        
    def _unfreeze_by_group(self, n_groups):
        """Helper for unfreeze() method."""
        length = len(self.groups)
        for i, group in enumerate(self.groups):
            setting = i >= length - n_groups
            for p in group.parameters():
                p.requires_grad = setting
                
    def __len__(self):
        """Number of parameter matrices in model (basically number of layers, 
        except that biases are counted separately).
        """
        return sum(1 for p in self.parameters())
    
    def dims(self):
        """Get shape of each layer's weights."""
        return [tuple(p.shape) for p in self.parameters()]

    def trainable(self):
        """Check which layers are trainable."""
        return [(tuple(p.shape), p.requires_grad) for p in self.parameters()]

    def weight_stats(self):
        """Check mean and standard deviation of each layer's weights."""
        return [stats(p.data, 3) for p in self.parameters()]

    def plot_weights(self):
        """Plot histograms of each layer's weights."""
        n_layers = len(self.dims())
        fig, ax = plt.subplots(n_layers, figsize=(8, n_layers * 1.25))
        if not isinstance(ax, Iterable): ax = [ax]
        for i, p in enumerate(self.parameters()):
            ax[i].hist(p.data.flatten())
            ax[i].set_title(f'Shape: {tuple(p.shape)} Stats: {stats(p.data)}')
        plt.tight_layout()
        plt.show()

In [759]:
class SimpleModel(BaseModel):
    
    def __init__(self, dim):
        super().__init__()  
        self.fc1 = nn.Linear(dim, 2)
        self.fc2 = nn.Linear(2, 1)
        
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        return self.fc2(x)

In [760]:
class GroupedModel(BaseModel):
    
    def __init__(self, dim):
        super().__init__()  
        g1 = nn.Sequential(
            nn.Linear(dim, 8),
            nn.LeakyReLU(),
            nn.Linear(8, 4),
            nn.LeakyReLU()
        )
        g2 = nn.Linear(4, 1)
        self.groups = nn.ModuleList([g1, g2])
        
    def forward(self, x):
        for group in self.groups:
            x = group(x)
        return x

In [761]:
snet = SimpleModel(DIM)
snet

SimpleModel(
  (fc1): Linear(in_features=2, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)

In [762]:
variable_lr_optimizer(snet, 1e-3)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.001
    weight_decay: 0
)

In [763]:
variable_lr_optimizer(snet, [3e-3, 1e-1])

/Users/harrisonmamin/ml_htools/ml_htools/torch_utils.py:203: UserWarning: More learning rates than layer groups. Last LRs will not be used.
  data = [{'params': group.parameters(), 'lr': lr}


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.003
    weight_decay: 0
)

In [764]:
snet.freeze()
for n in range(5):
    snet.unfreeze(n_layers=n)
    print(n, snet.trainable())

0 [((2, 2), False), ((2,), False), ((1, 2), False), ((1,), False)]
1 [((2, 2), False), ((2,), False), ((1, 2), False), ((1,), True)]
2 [((2, 2), False), ((2,), False), ((1, 2), True), ((1,), True)]
3 [((2, 2), False), ((2,), True), ((1, 2), True), ((1,), True)]
4 [((2, 2), True), ((2,), True), ((1, 2), True), ((1,), True)]


In [765]:
snet.freeze()
with assert_raises(AttributeError) as ar:
    for n in range(3):
        snet.unfreeze(n_groups=n)
        print(n, snet.trainable())

As expected, got AttributeError('SimpleModel' object has no attribute 'groups').


In [766]:
gnet = GroupedModel(DIM)
gnet

GroupedModel(
  (groups): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
    (1): Linear(in_features=4, out_features=1, bias=True)
  )
)

In [767]:
variable_lr_optimizer(gnet, [1e-3, 3e-3])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.001
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.003
    weight_decay: 0
)

In [768]:
variable_lr_optimizer(gnet, 1e-3)

/Users/harrisonmamin/ml_htools/ml_htools/torch_utils.py:199: UserWarning: Fewer learning rates than layer groups. Repeating last LR.
  warnings.warn('Fewer learning rates than layer groups. '


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.001
    weight_decay: 0

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 0.001
    lr: 0.001
    weight_decay: 0
)

In [769]:
gnet.freeze()
for n in range(3):
    gnet.unfreeze(n_groups=n)
    print(n, gnet.trainable())

0 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), False), ((1, 4), False), ((1,), False)]
1 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), False), ((1, 4), True), ((1,), True)]
2 [((8, 2), True), ((8,), True), ((4, 8), True), ((4,), True), ((1, 4), True), ((1,), True)]


In [770]:
gnet.freeze()
for n in range(7):
    gnet.unfreeze(n_layers=n)
    print(n, gnet.trainable())

0 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), False), ((1, 4), False), ((1,), False)]
1 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), False), ((1, 4), False), ((1,), True)]
2 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), False), ((1, 4), True), ((1,), True)]
3 [((8, 2), False), ((8,), False), ((4, 8), False), ((4,), True), ((1, 4), True), ((1,), True)]
4 [((8, 2), False), ((8,), False), ((4, 8), True), ((4,), True), ((1, 4), True), ((1,), True)]
5 [((8, 2), False), ((8,), True), ((4, 8), True), ((4,), True), ((1, 4), True), ((1,), True)]
6 [((8, 2), True), ((8,), True), ((4, 8), True), ((4,), True), ((1, 4), True), ((1,), True)]


# Callbacks

In [771]:
@auto_repr
class TorchCallback:
    
    def on_train_begin(self, trainer, lrs):
        pass
    
    def on_train_end(self, trainer, epoch, stats, val_stats):
        pass
    
    def on_epoch_begin(self, trainer, epoch, stats, val_stats):
        pass

    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        pass
    
    def on_batch_begin(self, trainer, i, sum_i, stats):
        pass
    
    def on_batch_end(self, trainer, i, sum_i, stats):
        pass

In [772]:
class BasicConfig(TorchCallback):
    """Handles basic model tasks like putting the model on the GPU
    and switching between train and eval modes.
    """
    
    def __init__(self, priority=0):
        self.priority = priority
    
    def on_train_begin(self, trainer, lrs):
        if not isinstance(lrs, Iterable): lrs = [lrs]
        trainer.net.to(DEVICE)
        update_optimizer(trainer.optim, *lrs)

    def on_epoch_begin(self, trainer, *args, **kwargs):
        trainer.net.train()
        
    def on_train_end(self, trainer, *args, **kwargs):
        trainer.logger.info('Training complete. Model in eval mode.')
        trainer.net.eval()

In [773]:
class StatsHandler(TorchCallback):
    """This updates metrics at the end of each epoch to account for
    potentially varying batch sizes.
    """
    
    def __init__(self, priority=5):
        self.priority = priority
        
    def on_epoch_begin(self, trainer, epoch, stats, val_stats):
        """Resets stats at the start of each epoch."""
        stats.clear()
        
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        """Computes (possibly weighted) averages of mini-batch stats
        at the end of each epoch.
        """
        for group in (stats, val_stats):
            for k, v in group.items():
                if k == 'batch_size': continue
                group[k] = np.average(v, weights=group['batch_size'])
            group.pop('batch_size')

In [774]:
class MetricPrinter(TorchCallback):
    """Prints metrics at the end of each epoch. This is one of the 
    default callbacks provided in BaseModel - it does not need to
    be passed in explicitly.
    """
    
    def __init__(self, priority=10):
        self.priority = priority
        
    def on_train_begin(self, trainer, *args, **kwargs):
        trainer.logger = trainer.get_logger(
            os.path.join(trainer.out_dir, 'train.log'),
            fmt='\n%(asctime)s\n %(message)s'
        )
    
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        data = [[k, v, val_stats[k]] for k, v in stats.items()]
        table = tabulate(data, headers=['Metric', 'Train', 'Validation'], 
                         tablefmt='github', floatfmt='.4f')
        trainer.logger.info(f'Epoch {epoch}\n\n{table}\n\n{"="*9}')

In [866]:
class EarlyStopper(TorchCallback):
    
    def __init__(self, goal, metric='loss', min_improvement=0.0, patience=3, 
                 priority=15):
        """
        Parameters
        ----------
        goal: str
            Indicates what we want to do to the metric in question.
            Either 'min' or 'max'. E.g. metric 'loss' should have goal 'min'
            while metric 'precision' should have goal 'max'.
        metric: str
            Quantity to monitor. This will always be computed on the 
            validation set.
        min_improvement: float
            Amount of change needed to qualify as improvement. For example,
            min_improvement of 0.0 means any improvement is sufficient. With
            a min_improvent of 0.2, we will stop training even if the
            quantity improves by, for example, 0.1.
        patience: int
            Number of acceptable epochs without improvement. E.g. patience=0 
            means the metric must improve every epoch for training to continue.            
        """
        # Will use op like: self.op(new_val, current_best)
        if goal == 'min':
            self.init_metric = self.best_metric = float('inf')
            self.op = lt
            self.op_best = sub
        elif goal == 'max':
            self.init_metric = self.best_metric = float('-inf')
            self.op = gt
            self.op_best = add
        else:
            raise ValueError('Goal must be "min" or "max".')
           
        self.priority = priority
        self.metric = metric
        self.min_improvement = min_improvement
        self.patience = patience
        self.since_improvement = 0
        
    def on_train_begin(self, trainer, *args, **kwargs):
        """Resets tracked variables at start of training."""
        self.best_metric = self.init_metric
        self.since_improvement = 0
    
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        # Error handling.
        new_val = val_stats.get(self.metric)
        if new_val is None:
            trainer.logger.info(f'EarlyStopper could not find {self.metric}.'
                                f'Callback behavior may not be enforced.')
        
        # Expected behavior.
        if self.op(new_val, self.op_best(self.best_metric, self.min_improvement)):
            self.best_metric = new_val
            self.since_improvement = 0
        else:
            self.since_improvement += 1
            if self.since_improvement > self.patience:
                trainer.logger.info(
                    f'EarlyStopper halting training: validation {self.metric} '
                    f'has not improved enough in {self.since_improvement} epochs.'
                )
                trainer._stop_training = True

In [867]:
class PerformanceThreshold(TorchCallback):
    
    def __init__(self, metric, goal, threshold, skip_epochs=0, split='val',
                 priority=15):
        assert split in ('train', 'val'), 'Split must be "train" or "val".'
        assert goal in ('min', 'max'), 'Goal must be "min" or "max"'
        
        self.priority = priority
        self.metric = metric
        self.threshold = threshold
        self.skip_epochs = skip_epochs
        self.split = split
        self.op = gt if goal == 'min' else lt
        
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        if epoch <= self.skip_epochs:
            return
        
        # Error handling.
        data = val_stats if self.split == 'val' else stats
        new_val = data.get(self.metric)
        if new_val is None:
            trainer.logger.info(f'{self.metric} not found in metrics.'
                                 'PerformanceThreshold may not be enforced.')
            return
        
        # Expected behavior.
        if self.op(new_val, self.threshold):
            trainer.logger.info(
                f'PerformanceThreshold halting training: {self.metric} '
                f'of {new_val:.4f} did not meet threshold.'
            )
            trainer._stop_training = True

In [881]:
class ModelCheckpoint(TorchCallback):
    
    def __init__(self, metric='loss', goal='min', priority=25):
        # Will use op like: self.op(new_val, current_best)
        if goal == 'min':
            self.init_metric = self.best_metric = float('inf')
            self.op = lt
            self.op_best = sub
        elif goal == 'max':
            self.init_metric = self.best_metric = float('-inf')
            self.op = gt
            self.op_best = add
        else:
            raise ValueError('Goal must be "min" or "max".')

        self.priority = priority
        self.metric = metric
        self.model_dir = None
        
    def on_train_begin(self, trainer, *args, **kwargs):
        self.best_metric = self.init_metric
        
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        new_val = val_stats.get(self.metric)
        # Error handling.
        if new_val is None:
            trainer.logger.info(f'{self.metric} not found in metrics.'
                                 'ModelCheckpoint may not save models.')
            
        # Expected behavior.
        if self.op(new_val, self.best_metric):
            trainer.logger.info(
                f'Saving model. {self.metric} improved from '
                f'{self.best_metric} to {new_val}.'
            )
            trainer.save(f'model.pkl')
            self.best_metric = new_val

In [882]:
class CSVLogger(TorchCallback):
    """Separate from StatsHandler in case we don't want to log outputs."""
    
    def __init__(self, mode='epoch', file_fmt='{}_stats.csv', priority=90):
        assert mode in ('epoch', 'batch'), \
            'Mode must be "epoch" or "batch".'
        
        self.priority = priority
        self.mode = mode
        self.history = defaultdict(list)
        self.fname = file_fmt.format(mode)
        
    def on_train_begin(self, trainer, lrs):
        pass
        
    def on_batch_end(self, trainer, i, sum_i, stats):
        if self.mode != 'batch':
            return
        
        pass
        
    def on_epoch_end(self, trainer, epoch, stats, val_stats):
        if self.mode != 'epoch':
            return
        
    def write_csv(self):
        pass

In [883]:
class S3Uploader(TorchCallback):
    """
    """
    
    def __init__(self, prefix, priority=95):
        self.prefix = prefix
        self.priority = priority
    
    def on_train_end(self, trainer, *args, **kwargs):
        paths = [f.path for f in os.scandir(trainer.out_dir)
                 if f.is_file() and not f.name.startswith('.')]
        s3 = S3tool()
        try:
            s3.upload_files(paths, trainer.bucket, self.prefix)
        except Exception as e:
            trainer.logger.error(e)

In [884]:
class EC2Closer(TorchCallback):
    
    def __init__(self, priority=100):
        self.priority = priority
        
    def on_train_end(self, trainer, *args, **kwargs):
        stop_instance()

In [885]:
class ModelUnfreezer(TorchCallback):
    """Gradually unfreeze a model during training.
    """
    
    def __init__(self, idx, mode='epoch', priority=25):
        self.priority = priority
        
    def on_batch_begin(self):
        pass
    
    def on_epoch_begin(self):
        pass

In [886]:
def back_translate(text, to, from_lang='en'):
    return TextBlob(text)\
        .translate(to=to)\
        .translate(from_lang=to, to=from_lang)

In [887]:
text = """
Visit ESPN to get up-to-the-minute sports news coverage, scores, highlights and commentary for NFL, MLB, NBA, College Football, NCAA Basketball and more.
"""
# back_translate(text, 'es')

# Metrics

Keep sklearn pattern with y_true as first argument.

For classification problems, round probabilities once instead of in every metric.

In [888]:
def hasarg(func, arg):
    """Checks if a function has a given argument. 
    Works with *args and **kwargs as well.
    
    Parameters
    ----------
    func: function
    arg: str
        Name of argument to look for.
    
    Returns
    -------
    bool
    """
    return arg in inspect.signature(func).parameters

In [889]:
def percent_positive(y_true, y_pred):
    """Compute the percent of predictions that are positive. This
    can help us identify when a model is predicting all ones or zeros.
    """
    return (y_pred == 1).float().mean()

In [890]:
def mean_soft_prediction(y_true, y_score):
    """Compute the mean predicted probability."""
    return y_score.mean() 

In [891]:
def batch_size(y_true, y_pred):
    """Count the number of items in the current batch."""
    return y_true.shape[0]

In [892]:
[hasarg(roc_auc_score, val) for val in ('y_score', 'y_pred')]

[True, False]

In [893]:
[hasarg(precision_score, val) for val in ('y_score', 'y_pred')]

[False, True]

# Training

In [894]:
DIM = 2
metrics = [accuracy_score, 
           precision_score, 
           recall_score, 
           percent_positive,
           mean_soft_prediction
          ]
callbacks = [EarlyStopper('max', 'accuracy', patience=3),
             PerformanceThreshold('recall', 'max', 0.25, skip_epochs=5),
             CSVLogger(),
             ModelCheckpoint(),
             S3Uploader('mytorch_test')]

In [895]:
train = Data(n=34, dim=DIM)
val = Data(n=30, dim=DIM)

dl_train = DataLoader(train, batch_size=8, shuffle=True)
dl_val = DataLoader(val, batch_size=8, shuffle=False)

In [898]:
t = Trainer(gnet, train, val, dl_train, dl_val, F.binary_cross_entropy_with_logits, 
            '../data/v1', 'datascience-delphi-dev', torch.optim.RMSprop, 
            last_act=torch.sigmoid, metrics=metrics, callbacks=callbacks)
t

Trainer(criterion='binary_cross_entropy_with_logits', out_dir='../data/v1', bucket='datascience-delphi-dev')

Datasets: 34 train rows, 30 val rows

Optimizer: RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 0.001
    lr: 0.003
    momentum: 0
    weight_decay: 0

Parameter Group 1
    alpha: 0.99
    centered: False
    eps: 0.001
    lr: 0.003
    momentum: 0
    weight_decay: 0
)

GroupedModel(
  (groups): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
    (1): Linear(in_features=4, out_features=1, bias=True)
  )
))

In [899]:
t.fit(10, [3e-2])


2020-02-12 16:15:09,397
 Epoch 1

| Metric               |   Train |   Validation |
|----------------------|---------|--------------|
| loss                 |  0.1176 |       0.0014 |
| accuracy             |  0.9412 |       1.0000 |
| precision            |  0.9706 |       1.0000 |
| recall               |  0.9664 |       1.0000 |
| percent_positive     |  0.6471 |       0.6333 |
| mean_soft_prediction |  0.6581 |       0.6331 |


2020-02-12 16:15:09,398
 Saving model. loss improved from inf to 0.0014312235619096706.
Data written to ../data/v1/model.pkl.

2020-02-12 16:15:09,430
 Epoch 2

| Metric               |   Train |   Validation |
|----------------------|---------|--------------|
| loss                 |  0.0222 |       0.0037 |
| accuracy             |  1.0000 |       1.0000 |
| precision            |  1.0000 |       1.0000 |
| recall               |  1.0000 |       1.0000 |
| percent_positive     |  0.6471 |       0.6333 |
| mean_soft_prediction |  0.6288 |       0.6365 |




/Users/harrisonmamin/ml_htools/ml_htools/torch_utils.py:231: UserWarning: Fewer learning rates than layer groups. Repeating last LR.
  warnings.warn('Fewer learning rates than layer groups. '
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/harrisonmamin/.pyenv/versions/3.7.4/envs/main/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
  0%|          | 0/2 [00:00<?, ?it/s]

File: ../data/v1/train.log
Bucket: datascience-delphi-dev
Key: mytorch_test/train.log


  0%|          | 0/2 [00:00<?, ?it/s]

File: ../data/v1/model.pkl
Bucket: datascience-delphi-dev
Key: mytorch_test/model.pkl


In [802]:
t.save('trainer.zip')

Data written to ../data/v1/trainer.zip.


In [803]:
t2 = Trainer.from_file(os.path.join('..', 'data', 'v1', 'trainer.zip'))
print(t2)
del t2; gc.collect()

Object loaded from ../data/v1/trainer.zip.
Trainer(criterion='binary_cross_entropy_with_logits', out_dir='../data/v1', bucket='datascience-delphi-dev')

Datasets: 34 train rows, 30 val rows

Optimizer: RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 0.001
    lr: 0.03
    momentum: 0
    weight_decay: 0

Parameter Group 1
    alpha: 0.99
    centered: False
    eps: 0.001
    lr: 0.03
    momentum: 0
    weight_decay: 0
)

GroupedModel(
  (groups): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=2, out_features=8, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
    (1): Linear(in_features=4, out_features=1, bias=True)
  )
))


1130